In [1]:
import pandas as pd
import os
import glob
import json

import nltk
import string
import contractions
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

## 1. Data Preprocessing

In [2]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

### 1.1 Concatenate comments files

In [3]:
comments_path = r'D:/Python/GA/WSB_comments//'
os.listdir(os.path.join(comments_path))

['20201201-1215comments.csv',
 '20201216-20210101comments.csv',
 '20210102-0115comments.csv',
 '20210116-0201comments.csv',
 '20210202-0210comments.csv',
 '20210211-0220comments.csv',
 '20210221-0301comments.csv',
 '20210302-0315comments.csv',
 '20210316-0401comments.csv',
 '20210402-0415comments.csv',
 '20210416-0430comments.csv']

In [4]:
pd.read_csv(comments_path+'\\20201201-1215comments.csv', index_col=None,header = 0)

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,body,collapsed_because_crowd_control,comment_type,created_utc,gildings,id,is_submitter,link_id,locked,no_follow,parent_id,permalink,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,created,author_cakeday,distinguished
0,[],NaN,picklerickerton,NaN,NaN,[],NaN,NaN,NaN,text,t2_ltraths,False,False,[],"every crude dip gets bought within the hour during market hours, its impressive",NaN,NaN,1607990399,{},gfv8d5f,False,t3_kd6fsb,False,False,t3_kd6fsb,/r/wallstreetbets/comments/kd6fsb/what_are_your_moves_tomorrow_december_15_2020/gfv8d5f/,1607995688,3,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.607990e+09,NaN,NaN
1,[],NaN,Cowboyre,NaN,NaN,"[{'e': 'text', 't': '\x1b201204:1:3'}]",NaN,201204:1:3,dark,richtext,t2_2ozp0itg,False,True,[],I hate all of my co workers is anyone hiring? I have no skills and I’m a pretty slow learner but...,NaN,NaN,1607990398,{},gfv8d3x,False,t3_kd6fsb,False,False,t3_kd6fsb,/r/wallstreetbets/comments/kd6fsb/what_are_your_moves_tomorrow_december_15_2020/gfv8d3x/,1607995687,18,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.607990e+09,NaN,NaN
2,[],NaN,OverpricedBagel,NaN,5b951270-9b43-11e8-8a57-0e532bbfe7e8,"[{'e': 'text', 't': 'Citron Research'}]",NaN,Citron Research,dark,richtext,t2_70z5dg1i,False,True,[],Fuck off boomer,NaN,NaN,1607990397,{},gfv8d03,False,t3_kd8zwk,False,True,t3_kd8zwk,/r/wallstreetbets/comments/kd8zwk/steel_stallions_unite/gfv8d03/,1607995685,-2,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.607990e+09,NaN,NaN
3,[],NaN,misterbuh,NaN,NaN,[],NaN,NaN,NaN,text,t2_1y6wk7yk,False,True,[],Entry level posting answering a phone doesn’t pay twenty five dollars and my statement is obviou...,NaN,NaN,1607990395,{},gfv8cvh,False,t3_kd7zao,False,False,t1_gfv6vi4,/r/wallstreetbets/comments/kd7zao/evictions_like_stonks_only_go_up/gfv8cvh/,1607995683,11,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.607990e+09,NaN,NaN
4,[],NaN,[deleted],NaN,NaN,NaN,NaN,NaN,dark,NaN,NaN,NaN,NaN,[],[removed],NaN,NaN,1607990389,{},gfv8cha,False,t3_kb57da,False,True,t3_kb57da,/r/wallstreetbets/comments/kb57da/royalty_pharma_rprx_lockup_expiration_final_call/gfv8cha/,1607995678,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.607990e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803505,[],NaN,epicoliver3,NaN,NaN,"[{'e': 'text', 't': '\x1b200918:30:1'}]",NaN,200918:30:1,dark,richtext,t2_1cz3fm8p,False,False,[],FUCK U 🏳️‍🌈🐻,NaN,NaN,1606780803,{},ge7etp5,False,t3_k45hxk,False,False,t1_ge6zz1w,/r/wallstreetbets/comments/k45hxk/gme_to_the_moon_ive_been_buying_and_holding_for_a/ge7etp5/,1606787245,8,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.606781e+09,NaN,NaN
803506,[],NaN,Red_leaf96,NaN,NaN,"[{'e': 'text', 't': '\x1b201125:3:2'}]",NaN,201125:3:2,dark,richtext,t2_12psyg,False,False,[],"Puts on Amazon. I ordered 500 bucks worth of tools, they claimed it was delivered to the recepti...",NaN,NaN,1606780803,{},ge7etp3,False,t3_k459ll,False,False,t3_k459ll,/r/wallstreetbets/comments/k459ll/what_are_your_moves_tomorrow_december_01_2020/ge7etp3/,1606787245,2,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.606781e+09,NaN,NaN
803507,[],NaN,bneffaz,NaN,NaN,[],NaN,NaN,NaN,text,t2_hkd3o,False,False,[],CHGG fuck yeah,NaN,NaN,1606780802,{},ge7etnb,False,t3_k459ll,False,True,t3_k459ll,/r/wallstreetbets/comments/k459ll/what_are_your_moves_tomorrow_december_01_2020/ge7etnb/,1606787244,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],1.606781e+09,NaN,NaN
803508,[],NaN,kidruzen,NaN,NaN,[],NaN,NaN,NaN,text,t2_4vs0a6a1,False,False,[],So the sp 500 will add tesla all at once on the 21st. Does that mean it will moon on t

In [5]:
def concat_cvs(path):
    files = glob.glob(os.path.join(path, '*.csv'))
    concat_df = pd.concat((pd.read_csv(file, index_col = None,
                                       header = 0, 
                                       low_memory=False,
                                       #nrows= 80000
                                      ) 
                           for file in files
                          ), 
                          ignore_index=True)
    
    return concat_df

In [6]:
comments = concat_cvs(comments_path)

In [7]:
len(pd.unique(comments['id']))

9590044

In [8]:
comments.shape

(11273454, 39)

### 1.2 Features Selection

In [9]:
def Choose_features(dataframe, features):
    choosed_features = dataframe[features]
    
    return choosed_features

In [10]:
choosed_features = ['author_fullname',
                    'body',
                    'created_utc',
                    'distinguished',
                    #'glided',
                    'id',
                    'is_submitter',
                    'link_id',
                    'parent_id',
                    'score']

In [11]:
comments = comments[choosed_features]
comments.head()

,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_ltraths,"every crude dip gets bought within the hour during market hours, its impressive",1607990399,NaN,gfv8d5f,False,t3_kd6fsb,t3_kd6fsb,3
1,t2_2ozp0itg,I hate all of my co workers is anyone hiring? I have no skills and I’m a pretty slow learner but...,1607990398,NaN,gfv8d3x,False,t3_kd6fsb,t3_kd6fsb,18
2,t2_70z5dg1i,Fuck off boomer,1607990397,NaN,gfv8d03,False,t3_kd8zwk,t3_kd8zwk,-2
3,t2_1y6wk7yk,Entry level posting answering a phone doesn’t pay twenty five dollars and my statement is obviou...,1607990395,NaN,gfv8cvh,False,t3_kd7zao,t1_gfv6vi4,11
4,NaN,[removed],1607990389,NaN,gfv8cha,False,t3_kb57da,t3_kb57da,1


### 1.3 Datatime Convertion

In [12]:
comments.loc[:,'created_utc'] = pd.to_datetime(comments.created_utc, unit='s')

In [13]:
comments.head()

,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_ltraths,"every crude dip gets bought within the hour during market hours, its impressive",2020-12-14 23:59:59,NaN,gfv8d5f,False,t3_kd6fsb,t3_kd6fsb,3
1,t2_2ozp0itg,I hate all of my co workers is anyone hiring? I have no skills and I’m a pretty slow learner but...,2020-12-14 23:59:58,NaN,gfv8d3x,False,t3_kd6fsb,t3_kd6fsb,18
2,t2_70z5dg1i,Fuck off boomer,2020-12-14 23:59:57,NaN,gfv8d03,False,t3_kd8zwk,t3_kd8zwk,-2
3,t2_1y6wk7yk,Entry level posting answering a phone doesn’t pay twenty five dollars and my statement is obviou...,2020-12-14 23:59:55,NaN,gfv8cvh,False,t3_kd7zao,t1_gfv6vi4,11
4,NaN,[removed],2020-12-14 23:59:49,NaN,gfv8cha,False,t3_kb57da,t3_kb57da,1


In [14]:
comments.shape

(11273454, 9)

### 1.4 Filter Data Based on "Pre", "During" and "Post" Stages

In [17]:
comments = pd.concat([comments[(comments['created_utc'] >= "2021-1-3") & (comments['created_utc'] <= "2021-1-24") ],
                     comments[(comments['created_utc'] >= "2021-1-25") & (comments['created_utc'] <= "2021-2-8") ],
                     comments[(comments['created_utc'] >= "2021-2-9") & (comments['created_utc'] <= "2021-3-1") ]],
                     ignore_index=True)
comments

,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_538xerwi,DKNG $56 wall got shredded this morning. Will they rebuild or is it ATH next time we break 🤔,2021-01-14 23:59:59,NaN,gjadva9,False,t3_kxenqf,t3_kxenqf,1
1,t2_3mitpczp,Gov Cuomo from New York [https://www.independent.co.uk/news/world/americas/us-politics/new-york-...,2021-01-14 23:59:59,NaN,gjadv9p,False,t3_kxenqf,t1_gjadhs3,1
2,t2_cky20,"I guess it pays to comb over hundreds of comments. But seriously, I wish i saw this. Props dude",2021-01-14 23:59:59,NaN,gjadv7w,False,t3_kwps6l,t1_gj7e9wd,2
3,NaN,[deleted],2021-01-14 23:59:58,NaN,gjadv6z,False,t3_kxenqf,t3_kxenqf,4
4,t2_rfq8g,Damn CNK is ramping tf up.,2021-01-14 23:59:58,NaN,gjadv6g,False,t3_kxenqf,t3_kxenqf,2
...,...,...,...,...,...,...,...,...,...
4373218,NaN,[removed],2021-02-21 00:00:07,NaN,go684bm,False,t3_lodpnu,t1_go62b9v,1
4373219,NaN,[removed],2021-02-21 00:00:06,NaN,go6847q,False,t3_loef9o,t3_loef9o,1
4373220,t2_6p5skgkh,"That's awesome man, wish I wasn't at - 80 lol woulda done the same,",2021-02-21 00:00:05,NaN,go6844v,False,t3_lob61p,t3_lob61p,1
4373221,NaN,[removed],2021-02-21 00:00:03,NaN,go6841d,False,t3_loef9o,t1_go5pqfb,1


## 2. Text Preprocessing

In [18]:
# Check Null Value
for col in comments.columns:
    print(col, comments[col].isnull().sum())

author_fullname 1032277
body 4
created_utc 0
distinguished 4312993
id 0
is_submitter 0
link_id 0
parent_id 0
score 0


In [19]:
comments = comments.fillna("")

In [20]:
for col in comments.columns:
    print(col, comments[col].isnull().sum())

author_fullname 0
body 0
created_utc 0
distinguished 0
id 0
is_submitter 0
link_id 0
parent_id 0
score 0


In [21]:
comments['body'] = comments['body'].astype(str)

### 2.1 Expanding Contractions, Convert to Lowercase, Tokenization, Removing Punctuations

In [22]:
from tqdm import tqdm
tqdm.pandas(desc = 'Progress: ')

In [23]:
def expand(dataframe, feature):
    punc = string.punctuation
    
    # Expanding Contractions
    dataframe[feature] = dataframe[feature].progress_apply(lambda x:[contractions.fix(word) for word in x.split()])
    
    # Convert back into strings
    dataframe[feature] = [' '.join(map(str, l)) for l in dataframe[feature]]
    
    return dataframe

In [24]:
expand(comments,'body')

Progress: 100%|██████████| 4373223/4373223 [01:26<00:00, 50581.88it/s]


,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_538xerwi,DKNG $56 wall got shredded this morning. Will they rebuild or is it ATH next time we break 🤔,2021-01-14 23:59:59,,gjadva9,False,t3_kxenqf,t3_kxenqf,1
1,t2_3mitpczp,Gov Cuomo from New York [https://www.independent.co.uk/news/world/americas/us-politics/new-york-...,2021-01-14 23:59:59,,gjadv9p,False,t3_kxenqf,t1_gjadhs3,1
2,t2_cky20,"I guess it pays to comb over hundreds of comments. But seriously, I wish i saw this. Props dude",2021-01-14 23:59:59,,gjadv7w,False,t3_kwps6l,t1_gj7e9wd,2
3,,[deleted],2021-01-14 23:59:58,,gjadv6z,False,t3_kxenqf,t3_kxenqf,4
4,t2_rfq8g,Damn CNK is ramping tf up.,2021-01-14 23:59:58,,gjadv6g,False,t3_kxenqf,t3_kxenqf,2
...,...,...,...,...,...,...,...,...,...
4373218,,[removed],2021-02-21 00:00:07,,go684bm,False,t3_lodpnu,t1_go62b9v,1
4373219,,[removed],2021-02-21 00:00:06,,go6847q,False,t3_loef9o,t3_loef9o,1
4373220,t2_6p5skgkh,"That is awesome man, wish I was not at - 80 lol would have done the same,",2021-02-21 00:00:05,,go6844v,False,t3_lob61p,t3_lob61p,1
4373221,,[removed],2021-02-21 00:00:03,,go6841d,False,t3_loef9o,t1_go5pqfb,1


In [25]:
def convert(dataframe, feature):
    punc = string.punctuation
    
    # Tokenization
    dataframe[feature] = dataframe[feature].progress_apply(word_tokenize)
    #dataframe[feature] = dataframe[feature].progress_apply(sent_tokenize)
    
    # Lowercase
    dataframe[feature] = dataframe[feature].progress_apply(lambda x: [word.lower() for word in x])
    
    # Removing Punctuations
    dataframe[feature] = dataframe[feature].progress_apply(lambda x: [word for word in x if word not in punc])
    
    return dataframe

In [26]:
convert(comments, 'body')

Progress: 100%|██████████| 4373223/4373223 [00:12<00:00, 362402.09it/s]


,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_538xerwi,"[dkng, 56, wall, got, shredded, this, morning, will, they, rebuild, or, is, it, ath, next, time,...",2021-01-14 23:59:59,,gjadva9,False,t3_kxenqf,t3_kxenqf,1
1,t2_3mitpczp,"[gov, cuomo, from, new, york, https, //www.independent.co.uk/news/world/americas/us-politics/new...",2021-01-14 23:59:59,,gjadv9p,False,t3_kxenqf,t1_gjadhs3,1
2,t2_cky20,"[i, guess, it, pays, to, comb, over, hundreds, of, comments, but, seriously, i, wish, i, saw, th...",2021-01-14 23:59:59,,gjadv7w,False,t3_kwps6l,t1_gj7e9wd,2
3,,[deleted],2021-01-14 23:59:58,,gjadv6z,False,t3_kxenqf,t3_kxenqf,4
4,t2_rfq8g,"[damn, cnk, is, ramping, tf, up]",2021-01-14 23:59:58,,gjadv6g,False,t3_kxenqf,t3_kxenqf,2
...,...,...,...,...,...,...,...,...,...
4373218,,[removed],2021-02-21 00:00:07,,go684bm,False,t3_lodpnu,t1_go62b9v,1
4373219,,[removed],2021-02-21 00:00:06,,go6847q,False,t3_loef9o,t3_loef9o,1
4373220,t2_6p5skgkh,"[that, is, awesome, man, wish, i, was, not, at, 80, lol, would, have, done, the, same]",2021-02-21 00:00:05,,go6844v,False,t3_lob61p,t3_lob61p,1
4373221,,[removed],2021-02-21 00:00:03,,go6841d,False,t3_loef9o,t1_go5pqfb,1


### 2.2 Remove Stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
stop_words = set(stopwords.words('english'))
comments['body'] = comments['body'].progress_apply(lambda x: [word for word in x if word not in stop_words])
comments

Progress: 100%|██████████| 4373223/4373223 [00:12<00:00, 356041.95it/s]


,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_538xerwi,"[dkng, 56, wall, got, shredded, morning, rebuild, ath, next, time, break, 🤔]",2021-01-14 23:59:59,,gjadva9,False,t3_kxenqf,t3_kxenqf,1
1,t2_3mitpczp,"[gov, cuomo, new, york, https, //www.independent.co.uk/news/world/americas/us-politics/new-york-...",2021-01-14 23:59:59,,gjadv9p,False,t3_kxenqf,t1_gjadhs3,1
2,t2_cky20,"[guess, pays, comb, hundreds, comments, seriously, wish, saw, props, dude]",2021-01-14 23:59:59,,gjadv7w,False,t3_kwps6l,t1_gj7e9wd,2
3,,[deleted],2021-01-14 23:59:58,,gjadv6z,False,t3_kxenqf,t3_kxenqf,4
4,t2_rfq8g,"[damn, cnk, ramping, tf]",2021-01-14 23:59:58,,gjadv6g,False,t3_kxenqf,t3_kxenqf,2
...,...,...,...,...,...,...,...,...,...
4373218,,[removed],2021-02-21 00:00:07,,go684bm,False,t3_lodpnu,t1_go62b9v,1
4373219,,[removed],2021-02-21 00:00:06,,go6847q,False,t3_loef9o,t3_loef9o,1
4373220,t2_6p5skgkh,"[awesome, man, wish, 80, lol, would, done]",2021-02-21 00:00:05,,go6844v,False,t3_lob61p,t3_lob61p,1
4373221,,[removed],2021-02-21 00:00:03,,go6841d,False,t3_loef9o,t1_go5pqfb,1


In [29]:
'''
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    #text = " ".join(text)
    
    return text
    
comments['body'] = comments['body'].progress_apply(remove_stopwords)
comments
'''

'\ndef remove_stopwords(text):\n    stop_words = set(stopwords.words(\'english\'))\n    text = [word for word in text if word not in stop_words]\n    #text = " ".join(text)\n    \n    return text\n    \ncomments[\'body\'] = comments[\'body\'].progress_apply(remove_stopwords)\ncomments\n'

### 2.3 Lemmatization

In [30]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [31]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [32]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [33]:
# https://www.programcreek.com/python/?code=zadewg%2FElection-Meddling%2FElection-Meddling-master%2Fdeploy.py
def get_wordnet_pos(tag):    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 
        #return None

In [34]:
def Lemmatize(dataframe, feature):
    # Description for each tag:
    # https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    dataframe[feature] = dataframe[feature].progress_apply(nltk.tag.pos_tag)
    
    # Get wordnet postag
    dataframe[feature] = dataframe[feature].progress_apply(lambda x: 
                                                           [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
    # Lemmatization
    dataframe[feature] = dataframe[feature].progress_apply(lambda x: 
                                                           [WordNetLemmatizer().lemmatize(word, tag) for word, tag in x])
    return dataframe

In [35]:
Lemmatize(comments, 'body')

Progress: 100%|██████████| 4373223/4373223 [01:08<00:00, 64186.39it/s] 


,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_538xerwi,"[dkng, 56, wall, get, shredded, morning, rebuild, ath, next, time, break, 🤔]",2021-01-14 23:59:59,,gjadva9,False,t3_kxenqf,t3_kxenqf,1
1,t2_3mitpczp,"[gov, cuomo, new, york, http, //www.independent.co.uk/news/world/americas/us-politics/new-york-c...",2021-01-14 23:59:59,,gjadv9p,False,t3_kxenqf,t1_gjadhs3,1
2,t2_cky20,"[guess, pay, comb, hundred, comment, seriously, wish, saw, prop, dude]",2021-01-14 23:59:59,,gjadv7w,False,t3_kwps6l,t1_gj7e9wd,2
3,,[delete],2021-01-14 23:59:58,,gjadv6z,False,t3_kxenqf,t3_kxenqf,4
4,t2_rfq8g,"[damn, cnk, ramp, tf]",2021-01-14 23:59:58,,gjadv6g,False,t3_kxenqf,t3_kxenqf,2
...,...,...,...,...,...,...,...,...,...
4373218,,[remove],2021-02-21 00:00:07,,go684bm,False,t3_lodpnu,t1_go62b9v,1
4373219,,[remove],2021-02-21 00:00:06,,go6847q,False,t3_loef9o,t3_loef9o,1
4373220,t2_6p5skgkh,"[awesome, man, wish, 80, lol, would, do]",2021-02-21 00:00:05,,go6844v,False,t3_lob61p,t3_lob61p,1
4373221,,[remove],2021-02-21 00:00:03,,go6841d,False,t3_loef9o,t1_go5pqfb,1


In [36]:
%who_ls DataFrame

['comments']

In [53]:
comments

,author_fullname,body,created_utc,distinguished,id,is_submitter,link_id,parent_id,score
0,t2_538xerwi,"[dkng, 56, wall, get, shredded, morning, rebuild, ath, next, time, break, 🤔]",2021-01-14 23:59:59,,gjadva9,False,t3_kxenqf,t3_kxenqf,1
1,t2_3mitpczp,"[gov, cuomo, new, york, http, //www.independent.co.uk/news/world/americas/us-politics/new-york-c...",2021-01-14 23:59:59,,gjadv9p,False,t3_kxenqf,t1_gjadhs3,1
2,t2_cky20,"[guess, pay, comb, hundred, comment, seriously, wish, saw, prop, dude]",2021-01-14 23:59:59,,gjadv7w,False,t3_kwps6l,t1_gj7e9wd,2
3,,[delete],2021-01-14 23:59:58,,gjadv6z,False,t3_kxenqf,t3_kxenqf,4
4,t2_rfq8g,"[damn, cnk, ramp, tf]",2021-01-14 23:59:58,,gjadv6g,False,t3_kxenqf,t3_kxenqf,2
...,...,...,...,...,...,...,...,...,...
4373218,,[remove],2021-02-21 00:00:07,,go684bm,False,t3_lodpnu,t1_go62b9v,1
4373219,,[remove],2021-02-21 00:00:06,,go6847q,False,t3_loef9o,t3_loef9o,1
4373220,t2_6p5skgkh,"[awesome, man, wish, 80, lol, would, do]",2021-02-21 00:00:05,,go6844v,False,t3_lob61p,t3_lob61p,1
4373221,,[remove],2021-02-21 00:00:03,,go6841d,False,t3_loef9o,t1_go5pqfb,1


In [62]:
pre = comments[(comments['created_utc'] >= "2021-1-3") & (comments['created_utc'] <= "2021-1-24")].reset_index().drop('index', axis = 1)
during = comments[(comments['created_utc'] >= "2021-1-25") & (comments['created_utc'] <= "2021-2-8") ].reset_index().drop('index', axis = 1)
post = comments[(comments['created_utc'] >= "2021-2-9") & (comments['created_utc'] <= "2021-3-1") ].reset_index().drop('index', axis = 1)

In [1]:
#pre.to_csv('D:/Python/GA/pre_comments_clean.csv')
#during.to_csv('D:/Python/GA/during_comments_clean.csv')
#post.to_csv('D:/Python/GA/post_comments_clean.csv')

In [ ]:
#comments = pd.read_csv('D:/Python/GA/WSB_comments_clean.csv')
#comments